<a href="https://colab.research.google.com/github/cesarade/IA-Generativa-ChatGPT-Midjourney/blob/main/Base_LLM_vs_Fine_Tuned_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Base LLMs vs Fine-tuned LLMs

## Enunciado del caso

En este caso práctico, se propone al alumno la implementación de un modelo base quee haya sido pre-entrenado (se recomienda T5) y si comparación con el mismo modelo depués de aplicarle Fine-tuned (se recomienda Flan-T5)

# Resolución del caso práctico

## 0. Instalación de librerías externas

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.2 MB/s eta 0:00:00


# 1. Selección de un LLM base pre-entrenado

Tal y como hemos visto en secciones anteriores, existe una gran variedad de LLMs base que podemos utilzar: [https://huggingface.co/t5-base](https://huggingface.co/t5-base)

Este LLM esta compuesto por 220 millones de parámetros y ha sido pre-entrenado en número elevado de conjuntos de datos: [https://huggingface.co/google-t5/t5-base#training-details](https://huggingface.co/google-t5/t5-base#training-details)


## Lectura del modelo y del tokenizador

In [ ]:
import accelerate
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizador_T5 = T5Tokenizer.from_pretrained('t5-base')

model_T5 = T5ForConditionalGeneration.from_pretrained('t5-base', device_map = 'auto')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
prompt = 'My name'

prompt_tokens = tokenizador_T5(prompt, return_tensors='pt').input_ids.to('cuda')

outputs = model_T5.generate(prompt_tokens, max_length =100)

print(tokenizador_T5.decode(outputs[0]))

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
text = """The Second World War (also written World War II)1 was a global military \
conflict that took place between 1939 and 1945. It involved most of the world's \
nations - including all the major powers, as well as virtually all European nations \
- grouped into two opposing military alliances: the Allies on the one hand, and the \
Axis Powers on the other. It was the greatest war in history, with more than 100 \
million military personnel mobilized and a state of total war in which the major \
contenders devoted all their economic, military and scientific capabilities to the \
service of the war effort, blurring the distinction between civilian and military \
resources."""

prompt = f"Summarize: {text}"

prompt_tokens = tokenizador_T5(prompt, return_tensors='pt').input_ids.to('cuda')

outputs = model_T5.generate(prompt_tokens, max_length =100)

print(tokenizador_T5.decode(outputs[0]))

In [ ]:
prompt = "What do you think of Mars?"

prompt_tokens = tokenizador_T5(prompt, return_tensors='pt').input_ids.to('cuda')

outputs = model_T5.generate(prompt_tokens, max_length =100)

print(tokenizador_T5.decode(outputs[0]))

In [ ]:
prompt = "Translate to Spanish: 'How are you?'"

prompt_tokens = tokenizador_T5(prompt, return_tensors='pt').input_ids.to('cuda')

outputs = model_T5.generate(prompt_tokens, max_length =100)

print(tokenizador_T5.decode(outputs[0]))

In [ ]:
review = """Love these plugs, have a few now. We use them to plug in lights and \
set timers to turn them on and off via a phone app. Easy to use and linked to \
the internet and apps. Good value for money."""

prompt = f"Sentiment? Review: {review}"

prompt_tokens = tokenizador_T5(prompt, return_tensors='pt').input_ids.to('cuda')

outputs = model_T5.generate(prompt_tokens, max_length =100)

print(tokenizador_T5.decode(outputs[0]))

In [ ]:
review1 = """Love these plugs, have a few now. We use them to plug in lights and \
set timers to turn them on and off via a phone app. Easy to use and linked to \
the internet and apps. Good value for money."""

review2 = """Tried and tried but could never get them to work right. Too bad \
I'm past my return date or they would have gone back."""

review3 = """A well-sized, reliable smart plug. The app is easy to use and set \
up, and works well. I used them to make several lamps. Everything works fine - \
no problems."""

review4 = """Great little product. Super easy to set up. Didn't even need to use \
the Alexa app to do so. Did it with my echo. Now I use it almost daily to turn on \
a light that was a pain to get to."""

review5 = """If I could give this zero stars I would. Plug wouldn’t connect. I \
had to keep connecting it and finally just gave up and returned it. Customer service \
was a complete waste of time."""

prompt = f"""
Review: {review1}
Sentiment: Positive

Review: {review2}
Sentiment: Negative

Review: {review3}
Sentiment: Positive

Review: {review5}
Sentiment:"""

prompt_tokens = tokenizador_T5(prompt, return_tensors='pt').input_ids.to('cuda')

outputs = model_T5.generate(prompt_tokens, max_length =100)

print(tokenizador_T5.decode(outputs[0]))